## Ondas de gravedad externas

Esta rutina permite visualizar la propagacion de ondas de gravedad externas en el caso 1D a partir de una condicion inicial en la superficie libre y de un fondo dado.

Dani Risaro

Abril 2022


In [ ]:
import numpy as np

#-----------------------------------------------------------------------
def lax_wendroff(dx, dy, dt, g, u, v, h, u_tendency, v_tendency):

   # This function performs one timestep of the Lax-Wendroff scheme
   # applied to the shallow water equations

   # First work out mid-point values in time and space
   
    uh = u*h
    vh = v*h

    h_mid_xt = 0.5*(h[1:,:]+h[0:-1,:]) - (0.5*dt/dx)*(uh[1:,:]-uh[0:-1,:])

    h_mid_yt = 0.5*(h[:,1:]+h[:,0:-1]) - (0.5*dt/dy)*(vh[:,1:]-vh[:,0:-1])

    Ux = uh*u+0.5*g*h**2.
    Uy = uh*v
    uh_mid_xt = 0.5*(uh[1:,:]+uh[0:-1,:]) - (0.5*dt/dx)*(Ux[1:,:]-Ux[0:-1,:])
    uh_mid_yt = 0.5*(uh[:,1:]+uh[:,0:-1]) - (0.5*dt/dy)*(Uy[:,1:]-Uy[:,0:-1])

    Vx = Uy
    Vy = vh*v+0.5*g*h**2.
    vh_mid_xt = 0.5*(vh[1:,:]+vh[0:-1,:]) - (0.5*dt/dx)*(Vx[1:,:]-Vx[0:-1,:])
    vh_mid_yt = 0.5*(vh[:,1:]+vh[:,0:-1]) - (0.5*dt/dy)*(Vy[:,1:]-Vy[:,0:-1])

   # Ahora utilizamos los valores en el punto medio para predecir los valores
   # en el próximo paso temporal

    h_new = h[1:-1,1:-1] \
      - (dt/dx)*(uh_mid_xt[1:,1:-1]-uh_mid_xt[0:-1,1:-1]) \
      - (dt/dy)*(vh_mid_yt[1:-1,1:]-vh_mid_yt[1:-1,0:-1])


    Ux_mid_xt = uh_mid_xt*uh_mid_xt/h_mid_xt + 0.5*g*h_mid_xt**2.
    Uy_mid_yt = uh_mid_yt*vh_mid_yt/h_mid_yt
    
    uh_new = uh[1:-1,1:-1] \
      - (dt/dx)*(Ux_mid_xt[1:,1:-1]-Ux_mid_xt[0:-1,1:-1]) \
      - (dt/dy)*(Uy_mid_yt[1:-1,1:]-Uy_mid_yt[1:-1,0:-1]) \
      + dt*u_tendency*0.5*(h[1:-1,1:-1]+h_new)


    Vx_mid_xt = uh_mid_xt*vh_mid_xt/h_mid_xt
    Vy_mid_yt = vh_mid_yt*vh_mid_yt/h_mid_yt + 0.5*g*h_mid_yt**2.
    vh_new = vh[1:-1,1:-1] \
      - (dt/dx)*(Vx_mid_xt[1:,1:-1]-Vx_mid_xt[0:-1,1:-1]) \
      - (dt/dy)*(Vy_mid_yt[1:-1,1:]-Vy_mid_yt[1:-1,0:-1]) \
      + dt*v_tendency*0.5*(h[1:-1,1:-1]+h_new)
    u_new = uh_new/h_new
    v_new = vh_new/h_new

    return (u_new, v_new, h_new)

#-----------------------------------------------------------------------

# Seccion 0: Definicion de parametros iniciales

GAUSSIAN_BLOB = 4
STEP = 5

# Possible orographies
FLAT = 0
SLOPE = 1


# ------------------------------------------------------------------
# Sección 1: configuracion
g = 9.81                # Aceleracion de la gravedad (m/s2)
f = 0#-1e-4               # Parametro de Coriolis (s-1)
beta = 0#1.6e-11          # Gradiente meridional de f (s-1m-1)

dt_mins              = 1.    # paso temporal (minutos)
output_interval_mins = 15.   # Tiempo entre outputs (minutos)
forecast_length_days = 1    # Largo de la simulacion (dias)

orography = FLAT
initial_conditions = STEP#GAUSSIAN_BLOB
initially_geostrophic = False    # Puede ser "True" o "False"
add_random_height_noise = False  # Puede ser "True" o "False"

nx = 301  # Cantidad de gridpoints zonales
ny = 201   # Cantidad de gridpoints meridionales

dx = 100e3   # Espaciamiento zonal entre puntos de grilla (m)
dy = dx           # Espaciamiento meridional entre puntos de grilla (m)

# Specify the range of heights to plot in metres
plot_height_range = np.array([9500., 10000.])

# ------------------------------------------------------------------
# Sección 2: Act on the configuration information
dt = dt_mins*60.0  # Paso temporal en (s)
output_interval = output_interval_mins*60.0  # Tiempo entre outputs en (s)
forecast_length = forecast_length_days*24.0*3600.0  # Largo de la simulacion en (s)
nt = int(np.fix(forecast_length/dt)+1)  # Cantidad de pasos temporales
timesteps_between_outputs = np.fix(output_interval/dt)
noutput = int(np.ceil(nt/timesteps_between_outputs))  # Cantidad de output frames

x = np.mgrid[0:nx]*dx  # Coordenada de distancia zonal (m)
y = np.mgrid[0:ny]*dy  # Coordenada de distancia meridional (m)
[Y,X] = np.meshgrid(y,x)  # Grilla de coordenadas espaciales

# Creamos el fondo H
if orography == FLAT:
    H = np.zeros((nx, ny))
elif orography == SLOPE:
    H = 9000.*2.*np.abs((np.mean(x)-X)/np.max(x))
else:
    print('Don''t know what to do with orography=' + np.num2str(orography))
    sys.exit()

# Creamos el campo inicial de H
if initial_conditions == GAUSSIAN_BLOB:
    std_blob = 8.0*dy  # Standard deviation of blob (m)
    height = 9750. + 1000.*np.exp(-((X-0.25*np.mean(x))**2.+(Y-np.mean(y))**2.)/(2.* \
                                                     std_blob**2.))
elif initial_conditions == STEP:
    height = 9750.*np.ones((nx, ny))
    height[np.where((X<np.max(x)/5.) & (Y>np.max(y)/10.) & (Y<np.max(y)*0.9))] = 9500.

else:
    print("Don't know what to do with initial_conditions=%f" % initial_conditions)
    sys.exit()


# Parámetro de coriolis con variacion en y
F = f+beta*(Y-np.mean(y))

# Inicializamos u y v en el reposo
u = np.zeros((nx, ny))
v = np.zeros((nx, ny))


# Definimos h como el espesor del fluido (y "height" va a ser la altura
# de la superficie libre)
h = height - H

# Inicializamos los 3D arrays donde se guardan los outputs
u_save = np.zeros((nx, ny, noutput))
v_save = np.zeros((nx, ny, noutput))
h_save = np.zeros((nx, ny, noutput))
t_save = np.zeros((noutput,1))

# Indice para guardar el output
i_save = 0

# ------------------------------------------------------------------
# Seccion 3: Loop principal

for n in range(nt):

    if np.mod(n,timesteps_between_outputs) == 0:

        max_u = np.sqrt(np.max(u[:]*u[:]+v[:]*v[:]))

        print("Time = %f hours (max %f)  max(|u|) = %f"
          % ((n)*dt/3600., forecast_length_days*24., max_u) )

        u_save[:,:,i_save] = u
        v_save[:,:,i_save] = v
        h_save[:,:,i_save] = h
        t_save[i_save] = (n)*dt
        i_save = i_save+1


   # Computamos las aceleraciones
    u_accel = F[1:-1,1:-1]*v[1:-1,1:-1] - (g/(2.*dx))*(H[2:,1:-1]-H[0:-2,1:-1])
    v_accel = -F[1:-1,1:-1]*u[1:-1,1:-1] - (g/(2.*dy))*(H[1:-1,2:]-H[1:-1,0:-2])

   # Con ell esquema Lax-Wendroff avanzamos un paso temporal
    (unew, vnew, h_new) = lax_wendroff(dx, dy, dt, g, u, v, h, u_accel, v_accel)

   # Actualizamos los cambios de viento y altura, taking care to enforce
   # boundary conditions

    u[1:-1,1:-1] = unew[0:,0:]
    u[[-1,0],1:-1]  = unew[[0,-1],:]
    u[1:-1,[0,-1]]  = unew[:,[0,-1]]

    v[1:-1,1:-1] = vnew[0:,0:]
    v[[-1,0],0]  = vnew[[0,-1],0]
    v[1:-1,[0,-1]]  = vnew[:,[0,-1]]

    v[:,[0, -1]] = 0.

    h[1:-1,1:-1] = h_new[0:,0:]
    h[[0,-1],1:-1]  = h_new[[-1,0],:]


# En esta seccion animamos el campo de altura producido por el modelo

import matplotlib.pyplot as plt

f = plt.figure(figsize=(12,7))
ax1 = f.add_subplot(111)

ax1.autoscale(enable=True, axis='y', tight=True)

# Pasamos las unidades de los ejes en miles de km (x e y estan en metros)
x_1000km = x * 1.e-6
y_1000km = y * 1.e-6

# Set colormap to have 64 entries
ncol = 64

# Interval between arrows in the velocity vector plot
interval = 6

# Set this to "True" to save each frame as a png file
plot_frames = False

# Decide whether to show height in metres or km
if np.mean(plot_height_range) > 1000:
    height_scale = 0.001
    height_title = 'Height (km)'
else:
    height_scale = 1
    height_title = 'Height (m)'


print('Maximum orography height = %f m' % np.max(H[:]))
u = np.squeeze(u_save[:,:,0])

# Loop through the frames of the animation
for it in range(noutput):

   # Extract the height and velocity components for this frame
    h = np.squeeze(h_save[:,:,it])
    u = np.squeeze(u_save[:,:,it])
    v = np.squeeze(v_save[:,:,it])

   # First plot the height field

   if it==0:

      # Plot the height field
        im = ax1.imshow(np.transpose(h+H)*height_scale, \
        extent=[np.min(x_1000km),np.max(x_1000km),np.min(y_1000km),np.max(y_1000km)], \
        cmap='RdBu_r')
      # Set other axes properties and plot a colorbar
        cb1 = f.colorbar(im,ax=ax1)
        cb1.set_label('height (m)')
      # Contour the terrain:
        cs = ax1.contour(x_1000km,y_1000km,np.flip(np.transpose(H), axis=0),levels=range(1,11001,100),colors='k')

      # Plot the velocity vectors
        Q = ax1.quiver(x_1000km[2::interval],y_1000km[2::interval], \
         np.transpose(u[2::interval,2::interval]), \
         np.transpose(v[2::interval,2::interval]), scale=5, scale_units='xy', pivot='mid')
         #scale=5e2,scale_units='xy',pivot='mid')
        ax1.set_ylabel('Y distance (1000s of km)')
        ax1.set_xlabel('X distance (1000s of km)')
        ax1.set_title(height_title)
        tx1 = ax1.text(0, np.max(y_1000km), 'Time = %.1f hours' % (t_save[it]/3600.))

    else:
      # top plot:
        im.set_data(np.transpose(H+h)*height_scale)
        cs.set_array(np.transpose(h))
        Q.set_UVC(np.transpose(u[2::interval,2::interval]), \
               np.transpose(v[2::interval,2::interval]))
        tx1.set_text('Time = %.1f hours' % (t_save[it]/3600.))


    im.set_clim((plot_height_range*height_scale))
    x_scale = np.linspace(0., np.max(x_1000km),16)

    ax1.axis((0., np.max(x_1000km), 0., np.max(y_1000km)))
    ax1.set_xticks(x_scale)

   # To make an animation we can save the frames as a
   # sequence of images
    if plot_frames:
        plt.savefig('frame%03d.png' % it,format='png')

    plt.pause(0.1)
